In [4]:
import pandas as pd
from enum import Enum

class ModificationType(Enum):
    MODIFY = 1
    ADD = 2
    DELETE = 3

dataset_s500 = pd.read_csv('c:\\CodeBook\\new_code_rework_TD\\commits_with_jira_sample_concsecutive_500_aggregated.csv')
dataset_s500['new_code'] = ''
dataset_s500['rework'] = ''

def calculate_new_code_and_rework(row):
    new_code = 0
    rework = 0
    
    #for sub_row in row.iterrows():
    change_type = dataset_s500['change_type']
    added_lines = dataset_s500['added_lines']
    deleted_lines = dataset_s500['deleted_lines']  

    new_code += max(0, added_lines - deleted_lines)
    rework += min(deleted_lines, added_lines)
        
    if change_type == ModificationType.ADD: # everything is new code because deleted lines are zero
        new_code += added_lines
    elif change_type == ModificationType.DELETE: # everything is rework because added lines are zero
        rework += deleted_lines #0
    elif change_type == ModificationType.MODIFY: # there can be both new code and rework 
        # check if added lines are zero
        if added_lines == 0:
            rework += deleted_lines
            #continue
        #check if deleted lines are zero
        if deleted_lines == 0:
            new_code += added_lines
            #continue
        elif added_lines == deleted_lines:
            rework += deleted_lines # all is rework (changed lines e.g. renaming something inside a method or class or inside a file)

        #  add 100, del 99 -> 1 new line, 99 rework. add 99, del 100 -> 99 rework, 

        elif added_lines > deleted_lines:
            new_code += added_lines - deleted_lines 
            rework += deleted_lines
        elif added_lines < deleted_lines:
            rework += deleted_lines - added_lines
            new_code += added_lines
                
    return new_code, rework

for row in dataset_s500.iterrows():
    if row[1]['change_type'] is not None:    
        new_code, rework = calculate_new_code_and_rework(row)
        dataset_s500.at[row[0], 'new_code'] = new_code
        dataset_s500.at[row[0], 'rework'] = rework
    if row.Index == 5:
        break

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().